In [1]:
from detoxify import Detoxify
from tqdm import tqdm
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
from wilds.common.grouper import CombinatorialGrouper
import torch

In [2]:
model = Detoxify('original', device='cuda')

In [3]:
dataset = get_dataset(dataset="civilcomments", download=True)

In [4]:
subset = "train"
data = dataset.get_subset(subset)
loader = get_train_loader("standard", data, batch_size=128)

In [5]:
meta_vars = [
    'male',
    'female',
    'LGBTQ',
    'christian',
    'muslim',
    'other_religions',
    'black',
    'white',
    'identity_any',
    'severe_toxicity',
    'obscene',
    'threat',
    'insult',
    'identity_attack',
    'sexual_explicit'
]
len(meta_vars)
interest_vars = ["male", "female", "black", "white"]
interest_ind = [meta_vars.index(v) for v in interest_vars]

In [6]:
predictions = []
labels = []
groups = []

for idx, batch in enumerate(tqdm(loader)):
    
    x, y, meta = batch
    pred = model.predict(x)["toxicity"]
    g = meta[:, interest_ind]

    predictions.extend(list(pred))
    labels.extend(y.tolist())
    groups.append(g)
    
#     grouper = CombinatorialGrouper(dataset, ["male"])
#     print(grouper.metadata_to_group(meta))
    
#     grouper = CombinatorialGrouper(dataset, ["female"])
#     print(grouper.metadata_to_group(meta))
    
#     grouper = CombinatorialGrouper(dataset, ["black"])
#     print(grouper.metadata_to_group(meta))
    
#     grouper = CombinatorialGrouper(dataset, ["white"])
#     print(grouper.metadata_to_group(meta))
    
#     print("g")
#     print(g)

#     if idx >= 20:
#         break

100%|████████████████████████████████████████████████████████████████| 2102/2102 [26:52<00:00,  1.30it/s]


In [7]:
X = torch.Tensor(predictions)
y = torch.Tensor(labels)
g = torch.concat(groups)

X.shape, y.shape, g.shape

(torch.Size([269038]), torch.Size([269038]), torch.Size([269038, 4]))

In [8]:
g.sum(0)

tensor([29810, 36244,  9896, 16698])

In [9]:
save_root = "../data/civil_comments/"
torch.save(X.cpu(), save_root+"X_{}.pt".format(subset))
torch.save(y.cpu(), save_root+"y_{}.pt".format(subset))
torch.save(g.cpu(), save_root+"g_{}.pt".format(subset))